## Installation

In [1]:
#pip install openpyxl

## Import 

In [2]:
import cv2
import os.path
import sys
from pathlib import Path
import time 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import math
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd 
from sklearn.metrics import pairwise
#import uuid

## Use right data sets  

In [3]:
labels = ['Lundi']
full_List_coordonate = []
background = None 

In [4]:
#If the project does not contain an image file 
if not os.path.exists('C:\\Users\\jujub\\CNN\\images'):
    os.makedirs('C:\\Users\\jujub\\CNN\\images')
    for label in labels:
        os.makedirs('C:\\Users\\jujub\\CNN\\images\\{}'.format(label))
elif not os.path.exists('Lundi.xlsx'):
    wb = Workbook()
    ws = wb.active
    
    ws['A1'] = 'NameMouv'
    ws['B1'] = 'SerieID'
    ws['C1'] = 'X1'
    ws['D1'] = 'Y1'
    ws['E1'] = 'X2'
    ws['F1'] = 'Y2'
    ws['G1'] = 'X3'
    ws['H1'] = 'Y3'
    ws['I1'] = 'Vector_X1'
    ws['J1'] = 'Vector_Y1'
    ws['K1'] = 'Vector_X2'
    ws['L1'] = 'Vector_Y2'
    ws['M1'] = 'Vector_X3'
    ws['N1'] = 'Vector_Y3'
    
    wb.save(filename = 'Lundi.xlsx')
    
else:
    pass

In [5]:
def convertArrayToImage(array):
    w, h = 512, 512
    data = np.zeros((h, w, 3), dtype=np.uint8)
    data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
    img = Image.fromarray(array, 'RGB')
    img.save('my.png')
    img.show()

In [6]:
#Hand 
class Hand(object):
    def __init__(self, frame, seg, top = 10 , right = 350):
        self.top = top
        self.right = right 
        self.frame = frame 
        self.seg = seg
        self.contours = []
        self.outline = self.draw_outline()  
        self.temp_List_Coor_final = []
        self.list_Coor_final = []
    
    def draw_outline(self, color=(0, 255, 0), thickness=2):
            cpy = self.frame.copy()
            #self.contours = cnts
            cv2.drawContours(cpy, [self.seg + (self.right, self.top)], -1, color, thickness)
            return cpy
    
    def getcoo(self):
        self.List_Coor_final= []
        self.list_finger_start = []
        self.list_finger_end = []
        convhull =  cv2.convexHull(self.seg, returnPoints=False)
        defects = cv2.convexityDefects(self.seg, convhull)
        if defects is not None:
            cnt = 0
            for i in range(defects.shape[0]): # calculate the angle
                s, e, f, d = defects[i][0]
                start = tuple(self.seg[s][0])
                end = tuple(self.seg[e][0])
                far = tuple(self.seg[f][0])
                a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  #      cosine theorem
                if angle <= np.pi / 2:  # angle less than 90 degree, treat as fingers
                    cnt += 1
                    end = self.coorConvert(end)
                    start = self.coorConvert(start)
                    self. list_finger_start.append(start)
                    self.list_finger_end.append(end) 
                    cv2.circle(self.frame , start, 5, (255, 0, 0), -1)
                    cv2.circle(self.frame , end, 5, (255, 0, 0), -1)
            self.List_Coor_final = self.findCoor()
        return self.frame
 
        
    def findCoor(self):
        temp= []
        for i in self.list_finger_start:
            ix, iy  = i
            for j in self.list_finger_end:
                jx, jy = j
                if jx-9<= ix<= jx+9 and jy-10<= iy<= jy+10:
                    cx = (ix + jx)/2
                    cy = (iy + jy)/2
                    temp.append(i)
                    temp.append(j)
                    self.temp_List_Coor_final.append((cx,cy))
        for reststart in self.list_finger_start:
            self.temp_List_Coor_final.append(reststart)
        for restend in self.list_finger_end:
            self.temp_List_Coor_final.append(restend)
        self.list_finger_start.clear()
        self.list_finger_end.clear()
        for elem in temp:
            #print(elem)
            #print(self.temp_List_Coor_final)
            if elem in self.temp_List_Coor_final:
                self.temp_List_Coor_final.remove(elem)
#             else:
#                 print(elem)
#                 print(self.temp_List_Coor_final)
#                 elem = elem[1]+1
#                 self.temp_List_Coor_final.remove(elem)
#         print(self.temp_List_Coor_final )
        self.list_Coor_final = self.temp_List_Coor_final.copy()
        #print(self.list_Coor_final)
        self.temp_List_Coor_final.clear()
        return self.frame       
                
    def coorConvert(self, tuples):
        cx, cy = tuples 
        cx = int(cx + self.right)
        cy = int(cy + self.top)
        return (cx,cy)
    
    def get_center_of_mass(self):
        corcenter = ()
        if len(self.seg) == 0:
            return None
        M = cv2.moments(self.seg)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            corcenter = (cX, cY)
            corcenter = self.coorConvert(corcenter)
            cv2.circle(self.frame, corcenter , 4, [255, 0, 0], -1)
        return self.frame, corcenter
    
    def dertermineFinger(self):
        _,coorCenter = self.get_center_of_mass()
        print(coorCenter)
        if coorCenter:
            cv2.circle(self.frame , coorCenter, 5, (0, 255, 0), -1)
        self.listDistances = []
        for i in self.list_Coor_final:
            distances = pairwise.euclidean_distances([coorCenter], Y=[i])[0]
        self.list_Coor_final.append(coorCenter)
        self.list_Coor_final.sort(key=lambda tup: tup[0])
        return self.frame , self.list_Coor_final
        

In [7]:
def run_avg(frame, aWeight):
    global background
    if background is None:
        background = frame.copy().astype("float")
        return background
    background = cv2.accumulateWeighted(frame, background, 0.5)
    
def contouring(frame, threshold=25):
    global background
    diff = cv2.absdiff(background.astype("uint8"), frame)
    threshold = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
    (cnts, _) = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return None 
    else:
        segment = max(cnts, key=cv2.contourArea)
        return (threshold, segment)

In [8]:
#saved the data in an excel file
def record_coordonate(temp_list_coor, label, numvid, full_List_coordonate):
    temp = []
    count = 0
    if label =='Lundi':
        Name_Mouv = 1
    elif label == 'Mardi':
        Name_Mouv = 2
    elif label == 'Vendredi':
        Name_Mouv = 3
    for i in temp_list_coor:
        i+= (Name_Mouv, count)
        temp.append(i)
        count += 1
    return temp

In [9]:
def videoCapture(labels,  full_List_coordonate, createData = False):
    if createData == True:
        temp_list_coordonate = []
        cap = cv2.VideoCapture(0)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        capture_duration = 3 + 3
        num_video = 1
        top, right, bottom, left = 10, 350, 225, 590
        for label in labels: 
            print('collecting for {}'.format(label))
            save_path = "C:\\Users\\jujub\\CNN\\images\\{}\\".format(label)
            for numvid in range(num_video):
                print ('remets')
                start_time = time.time()
                file_name = label + '_' + str(numvid)+'.mp4'
                save_path = save_path + file_name
                out = cv2.VideoWriter(save_path,fourcc, 30.0, (640,480))
                count = 0 
                num_frames = 0
                while(int(time.time() - start_time) < capture_duration ):
                    re,frame = cap.read()
                    frame = imutils.resize(frame, width=700)
                    frame = cv2.flip(frame, 1)
                    (height, width) = frame.shape[:2]
                    roi = frame[top:bottom, right:left]
                    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                    gray = cv2.GaussianBlur(gray, (7, 7), 0)
                    if re == True:
                        if num_frames < 30:
                            run_avg(gray, 0.5)
                        else:
                            cont = contouring(gray)
                            if cont is not None:
                                (tresh, seg) = cont 
                                hand = Hand(frame, seg)
                                frame = hand.getcoo()
                                frame = hand.draw_outline()
                                frame, temp_list_coor  = hand.dertermineFinger()
                                if len(temp_list_coor)== 3:
                                    temp_list_coordonate.append(temp_list_coor)
                                cv2.imshow('tresh', tresh)
                        frame = cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
                        num_frames += 1
                        cv2.imshow('frame', frame)
                        out.write(frame)
                        count += 1 
                        k = cv2.waitKey(5)
                        # Press 'q' to exit
                        if k == ord('q'):
                            break 
                    else:
                        break
                if save_path:
                    save_path = save_path.rpartition('\\')[0]+save_path.rpartition('\\')[1]
                print('enlève')
                #print(temp_list_coordonate)
                full_List_coordonate += record_coordonate(temp_list_coordonate, label, numvid, full_List_coordonate)
                #print(full_List_coordonate)
                temp_list_coordonate.clear()
                time.sleep(2)
        #When everything done, release the capture
        cap.release()
        out.release()
        cv2.destroyAllWindows()
    else:
        pass 
    return full_List_coordonate

In [10]:
full_List_coordonate = videoCapture(labels, full_List_coordonate, createData = True)
print(full_List_coordonate)

collecting for Lundi
remets
(498, 217)
(508, 198)
(517, 194)
(524, 191)
(524, 191)
(528, 187)
(531, 183)
(532, 180)
(534, 178)
(536, 175)
(528, 177)
(529, 175)
(530, 173)
(531, 171)
(533, 168)
(534, 165)
(535, 163)
(535, 161)
(535, 159)
(534, 158)
(532, 157)
(529, 157)
(527, 157)
(524, 158)
(521, 159)
(518, 161)
(515, 163)
(512, 165)
(510, 168)
(509, 171)
(508, 173)
(509, 175)
(510, 177)
(512, 178)
(514, 178)
(514, 178)
(518, 179)
(527, 178)
(540, 171)
(536, 175)
(541, 174)
(545, 172)
(548, 170)
(548, 170)
(550, 167)
(552, 164)
(553, 160)
(553, 157)
(553, 155)
(552, 151)
enlève
[[(434, 203), (485, 87), (528, 177), 1, 0], [(436, 198), (485, 84), (529, 175), 1, 1], [(438, 194), (487, 78), (530, 173), 1, 2], [(441, 186), (489, 73), (531, 171), 1, 3], [(442, 182), (491, 69), (533, 168), 1, 4], [(443, 178), (493, 63), (534, 165), 1, 5], [(444, 172), (492, 61), (535, 163), 1, 6], [(444, 168), (491, 60), (535, 161), 1, 7], [(442, 166), (491, 54), (535, 159), 1, 8], [(439, 164), (487, 53), (53

In [11]:
def saveDataSet(full_List_coordonate):
    wb = load_workbook('Lundi.xlsx')
    ws = wb['Sheet']
    row = ws.max_row + 1
    col = 1
    
    for x in range(len(full_List_coordonate)):
        ws.cell(row= row, column= col).value = full_List_coordonate[x][3]
        ws.cell(row= row, column= col+1).value = full_List_coordonate[x][4]
        ws.cell(row= row, column= col+2).value = full_List_coordonate[x][0][0]
        ws.cell(row= row, column= col+3).value = full_List_coordonate[x][0][1]
        ws.cell(row= row, column= col+4).value = full_List_coordonate[x][1][0]
        ws.cell(row= row, column= col+5).value = full_List_coordonate[x][1][1]
        ws.cell(row= row, column= col+6).value = full_List_coordonate[x][2][0]
        ws.cell(row= row, column= col+7).value = full_List_coordonate[x][2][1]
        row+=1
    wb.save('Lundi.xlsx')

def addVector(df):
    book = load_workbook('Lundi.xlsx')
    ws = book['Sheet']
    dfVector = df[["Vector_X1",'Vector_Y1','Vector_X2','Vector_Y2','Vector_X3','Vector_Y3']]
    writer = pd.ExcelWriter('Lundi.xlsx', engine = 'openpyxl')
    writer.book  = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    max_row = max((c.row for c in ws['I'] if c.value is not None))
    dfVector.to_excel(writer, sheet_name='Sheet', startrow = max_row, startcol = 8,
                      columns = ["Vector_X1",'Vector_Y1','Vector_X2','Vector_Y2','Vector_X3','Vector_Y3'], index = False,header= False)
    writer.save()
    writer.close()

In [12]:
def createVector(df):
#     #listMouv = list(dict.fromkeys(df['NameMouv']))
    grouped = df.groupby('NameMouv')
    df['Vector_X1'] = df['X1'].diff()
    df['Vector_Y1'] = df['Y1'].diff()
    df['Vector_X2'] = df['X2'].diff()
    df['Vector_Y2'] = df['Y2'].diff()
    df['Vector_X3'] = df['X3'].diff()
    df['Vector_Y3'] = df['Y3'].diff()
    print(df)
    return df

def removeDuplicateExcel():
    file_df = pd.read_excel("Lundi.xlsx")
    file_df_first_record = file_df.drop_duplicates(subset=["NameMouv",'SerieID', "X1","Y1","X2","Y2","X3","Y3"], keep="last")
    file_df_first_record.to_excel("Lundi.xlsx", index=False)

In [13]:
saveDataSet(full_List_coordonate)
dt = pd.read_excel('Lundi.xlsx',dtype={'NameMouv' : str, 
                  'SerieID' : int ,'X1': int, 'Y1': int,'X2': int, 'Y2': int ,'X3': int, 'Y3': int })
df = pd.DataFrame(dt)
df = createVector(df)

   NameMouv  SerieID   X1   Y1   X2  Y2   X3   Y3  Vector_X1  Vector_Y1  \
0         1        0  434  203  485  87  528  177        NaN        NaN   
1         1        1  436  198  485  84  529  175        2.0       -5.0   
2         1        2  438  194  487  78  530  173        2.0       -4.0   
3         1        3  441  186  489  73  531  171        3.0       -8.0   
4         1        4  442  182  491  69  533  168        1.0       -4.0   
5         1        5  443  178  493  63  534  165        1.0       -4.0   
6         1        6  444  172  492  61  535  163        1.0       -6.0   
7         1        7  444  168  491  60  535  161        0.0       -4.0   
8         1        8  442  166  491  54  535  159       -2.0       -2.0   
9         1        9  439  164  487  53  534  158       -3.0       -2.0   
10        1       10  435  162  483  50  532  157       -4.0       -2.0   
11        1       11  431  159  478  49  529  157       -4.0       -3.0   
12        1       12  425

In [14]:
addVector(df)
removeDuplicateExcel()

In [15]:
# file_df = pd.read_excel("DataSet.xlsx")
# indexNames = file_df[ (file_df['Vector_X1'] == 0) & (file_df['Vector_X2'] == 0) & (file_df['Vector_X3'] == 0)& (file_df['Vector_Y1'] == 0)& (file_df['Vector_Y2'] == 0)& (file_df['Vector_Y3'] == 0)].index
# file_df.drop(indexNames , inplace=True)
# file_df.to_excel("DataSet.xlsx", index=False)